In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import gmaps
from pprint import pprint
import requests
import json

import warnings; warnings.simplefilter('ignore')

import pymongo

In [2]:
from config import gmaps_key
gmaps.configure(api_key=gmaps_key)

In [3]:
tour_file = pd.read_csv('tour_schedule.csv')
tour_file

,Tour,City
0,Spring 2007,"Farmington, New Mexico, USA"
1,Spring 2007,"Tucson, Arizona, USA"
2,Spring 2007,"Dallas, Texas, USA"
3,Spring 2007,"Baton Rouge, Louisiana, USA"
4,Spring 2007,"New Orleans, Louisiana, USA"
...,...,...
458,Fall 2019,"Wartenberg, Germany"
459,Spring 2020,"Denver, Colorado, USA"
460,Spring 2020,"Lafayette, Colorado, USA"
461,Spring 2020,"Flagstaff, Arizona, USA"


In [4]:
tour_file['State'] = ''
tour_file['Country'] = ''
tour_file['Cast'] = ''
for index, row in tour_file.iterrows():
    splitd = row['City'].split(', ')
    if len(splitd) == 1:
        tour_file.loc[index, 'State'] = '-'
        tour_file.loc[index, 'City'] = splitd[0]
        tour_file.loc[index, 'Country'] = splitd[0]
    elif len(splitd) == 2:
        tour_file.loc[index, 'State'] = '-'
        tour_file.loc[index, 'City'] = splitd[0]
        tour_file.loc[index, 'Country'] = splitd[1]
    elif len(splitd) == 3:
        tour_file.loc[index, 'State'] = splitd[1]
        tour_file.loc[index, 'City'] = splitd[0]
        tour_file.loc[index, 'Country'] = splitd[2]
    splitt = row['Tour'].split(' ')
    if splitt[0] == 'Spring':
        cast = 'A'
    elif splitt[0] == 'Fall':
        cast = 'B'
    tour_file.loc[index, 'Cast'] = cast + splitt[1][-2] + splitt[1][-1] 
tour_file.head()

,Tour,City,State,Country,Cast
0,Spring 2007,Farmington,New Mexico,USA,A07
1,Spring 2007,Tucson,Arizona,USA,A07
2,Spring 2007,Dallas,Texas,USA,A07
3,Spring 2007,Baton Rouge,Louisiana,USA,A07
4,Spring 2007,New Orleans,Louisiana,USA,A07


In [10]:

url = f'https://maps.googleapis.com/maps/api/geocode/json?key={gmaps_key}'
# cast_tour['Lat'] = ''
# cast_tour['Long'] = ''
for index, row in tour_file.iterrows():
    if row['State'] != '-':
        append_url = f'&address=' + str(row['City']) + '%20' + str(row['State']) + '%20' + str(row['Country'])
    else:
        append_url = f'&address=' + str(row['City']) + '%20' + str(row['Country'])
    response = requests.get(url+append_url).json()
    tour_file.loc[index, 'Lat'] = response['results'][0]['geometry']['location']['lat']
    tour_file.loc[index, 'Long'] = response['results'][0]['geometry']['location']['lng']
    
tour_file.head()

,Tour,City,State,Country,Cast,Lat,Long
0,Spring 2007,Farmington,New Mexico,USA,A07,36.728058,-108.218686
1,Spring 2007,Tucson,Arizona,USA,A07,32.222607,-110.974711
2,Spring 2007,Dallas,Texas,USA,A07,32.776664,-96.796988
3,Spring 2007,Baton Rouge,Louisiana,USA,A07,30.451468,-91.187147
4,Spring 2007,New Orleans,Louisiana,USA,A07,29.951066,-90.071532


In [17]:
# Connecting to MongoDB to save dataset
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.uwp_db

tour_schedule = db.tour_schedule

tour_schedule.insert_many(tour_file.to_dict('records'))

In [12]:
cast = input('What cast did you travel in?')
cast_tour = tour_file.loc[tour_file['Cast'] == cast]

What cast did you travel in?B18


In [13]:
# url = f'https://maps.googleapis.com/maps/api/geocode/json?key={gmaps_key}'
# # cast_tour['Lat'] = ''
# # cast_tour['Long'] = ''
# for index, row in cast_tour.iterrows():
#     if row['State'] != '-':
#         append_url = f'&address=' + str(row['City']) + '%20' + str(row['State']) + '%20' + str(row['Country'])
#     else:
#         append_url = f'&address=' + str(row['City']) + '%20' + str(row['Country'])
#     response = requests.get(url+append_url).json()
#     cast_tour.loc[index, 'Lat'] = response['results'][0]['geometry']['location']['lat']
#     cast_tour.loc[index, 'Long'] = response['results'][0]['geometry']['location']['lng']
    
        
cast_tour

,Tour,City,State,Country,Cast,Lat,Long
414,Fall 2018,Denver,Colorado,USA,B18,39.739236,-104.990251
415,Fall 2018,Grand Island,Nebraska,USA,B18,40.926396,-98.342012
416,Fall 2018,North Platte,Nebraska,USA,B18,41.140276,-100.760145
417,Fall 2018,Sioux Falls,South Dakota,USA,B18,43.546022,-96.731265
418,Fall 2018,Salina,Kansas,USA,B18,38.840280,-97.611424
419,Fall 2018,Stillwater,Oklahoma,USA,B18,36.115607,-97.058368
420,Fall 2018,Houston,Texas,USA,B18,29.760427,-95.369803
421,Fall 2018,Torreón,-,Mexico,B18,25.542844,-103.406786
422,Fall 2018,Durango City,-,Mexico,B18,24.027720,-104.653176
423,Fall 2018,Zacatecas City,-,Mexico,B18,22.764198,-102.586556


In [14]:
marker_locations = cast_tour[['Lat', 'Long']]

info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>State</dt><dd>{State}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
tour_info = [info_box_template.format(**row) for index, row in cast_tour.iterrows()]
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, info_box_content=tour_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))